In [7]:
using Pkg;
#Pkg.add("DFTK")
Pkg.update()


    Updating registry at `~/.julia/registries/General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Installed DiffRules ─────────── v1.12.2


   Installed Parsers ───────────── v2.5.1
   Installed DiffResults ───────── v1.1.0
   Installed Spglib ────────────── v0.6.1
   Installed JuliaInterpreter ──── v0.9.16
   Installed Libxc_jll ─────────── v6.0.0+0


   Installed MPI ───────────────── v0.20.4


...

  ✗ ParallelTemperingMonteCarlo


  64 dependencies successfully precompiled in 88 seconds. 48 already precompiled.
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package
┌ Info: We haven't cleaned this depot up for a bit, running Pkg.gc()...
└ @ Pkg /cache/build/default-amdci4-3/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/Pkg.jl:719
      Active

 manifest files: 3 found
      Active

 artifact files: 101 found
      Active scratchspaces: 3 found
     Deleted

 no artifacts, repos, packages or scratchspaces


In [10]:
using ParallelTemperingMonteCarlo
using DelimitedFiles
using StaticArrays
using Random#, Plots


In [34]:
Pkg.add("BenchmarkTools")
using BenchmarkTools

   Resolving package versions...


   Installed BenchmarkTools ─ v1.3.2


    Updating `~/Code/ParallelTemperingMonteCarlo.jl/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
    Updating `~/Code/ParallelTemperingMonteCarlo.jl/Manifest.toml`


  [6e4b80f9] + BenchmarkTools v1.3.2
  [9abbd945] + Profile


Precompiling 

project...


  ✓ BenchmarkTools


  ✓ ParallelTemperingMonteCarlo
  2 dependencies successfully precompiled in 12 seconds. 110 already precompiled. 2 skipped during auto due to previous errors.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version


In [44]:
genvector1(max_displacement) = max_displacement * (rand(-0.5:0.001:0.5,3))

genvector2(max_displacement) = SVector((rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement)

genvector2 (generic function with 2 methods)

In [45]:
@benchmark genvector1(rand())

BenchmarkTools.Trial: 10000 samples with 585 evaluations.
 Range (min … max):  202.564 ns …  57.504 μs  ┊ GC (min … max): 0.00% … 75.57%
 Time  (median):     258.803 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   309.658 ns ± 770.045 ns  ┊ GC (mean ± σ):  5.89% ±  3.15%

  █▅▄▃▂ ▆▅▃▅▃▂▃▃▂▄▃▁▂▁   ▂▄▃▂▁▁                                 ▂
  ███████████████████████████████████▆▇▇▆▆▇▇▅▆▆▅▆▅▅▃▅▅▅▅▄▃▄▄▅▂▅ █
  203 ns        Histogram: log(frequency) by time        703 ns <

 Memory estimate: 160 bytes, allocs estimate: 2.

In [46]:
@benchmark genvector2(rand())

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):   9.309 ns … 215.115 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.812 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.057 ns ±   6.019 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇▇▁▄    ▂█▆▂▂▄▁  ▄▃     ▄                                    ▂
  ████▆▅▅▅███████▆▆██▇▅▆▅██▇▆▇▆▆▇▇▅▅▇▅▆▅▆▆▅▅▆▅▅▄▅▅▄▅▄▄▄▃▅▅▄▃▄▄ █
  9.31 ns       Histogram: log(frequency) by time      25.8 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.